In [2]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.api as sm

In [3]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [4]:
funding_liquidity, market_liquidity = liquidity_observables()

tsmom_model.py:412: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  funding_liquidity['TED Spread']=ted.resample(rule='m',how='last')/100.
tsmom_model.py:413: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  funding_liquidity['LIBOR term repo']=libor_term_repo.resample(rule='m',how='last')/100.
tsmom_model.py:417: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  PS=pd.read_csv('PS.csv',index_col=0,parse_dates=True).resample(rule='m',how='last')
tsmom_model.py:418: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  us10y_zero=quandl.get('FED/SVENY',authtoken=token).SVENY10.resample(rule='d',how='last').dropna()
tsmom_model.py:419: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  yield10=quandl.get('USTREASURY/YIELD',authtoken=token)['10 YR'].resample(rule='d',how='last').dropna()
tsmom_

In [8]:
factors=get_all_factors()

In [15]:
results_df=pd.DataFrame()
for reg_fact in funding_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=funding_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
results_df=results_df.set_index(factors.columns)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [17]:
results_df.round(2).T

,Amihud Annual,FHT Annual,Amihud Monthly,FHT Monthly
TED Spread,-0.23,-1.17,-0.34,-0.76
TED SpreadTstats,-0.63,-2.69,-0.95,-0.95
LIBOR term repo,0.26,0.09,0.25,0.38
LIBOR term repoTstats,2.04,1.33,1.96,1.69


In [68]:
results_df=pd.DataFrame()
for reg_fact in funding_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=funding_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [69]:
results_df.round(2)

,TED Spread,TED SpreadTstats,LIBOR term repo,LIBOR term repoTstats
0,-0.63,-0.41,0.49,1.50
1,-0.23,-0.63,0.26,2.04
2,-0.12,-0.21,-0.23,-3.48
3,2.03,0.72,0.76,2.14
4,1.07,1.89,0.26,0.82
5,-0.58,-0.71,-0.30,-0.97
6,0.16,0.50,0.11,0.91


In [70]:
#results_df=pd.DataFrame()
for reg_fact in market_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=market_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
results_df.set_axis(factors.columns)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  


In [19]:
print results_df.round(2).T.to_latex()

\begin{tabular}{lrrrr}
\toprule
{} &  Amihud Annual &  FHT Annual &  Amihud Monthly &  FHT Monthly \\
\midrule
TED Spread            &          -0.23 &       -1.17 &           -0.34 &        -0.76 \\
TED SpreadTstats      &          -0.63 &       -2.69 &           -0.95 &        -0.95 \\
LIBOR term repo       &           0.26 &        0.09 &            0.25 &         0.38 \\
LIBOR term repoTstats &           2.04 &        1.33 &            1.96 &         1.69 \\
\bottomrule
\end{tabular}

